# Setting

In [1]:
# requirements
!pip install emoji
!pip install beautifulsoup4
!pip install requests

     |████████████████████████████████| 133kB 5.5MB/s 


In [2]:
import pandas as pd
import numpy as np
import pickle
import time
import pickle
import requests
from bs4 import BeautifulSoup as bs 
import emoji
from sklearn.metrics import ndcg_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# my path: drive/MyDrive/Project/Yektanet/product_recommender
FOLDERNAME = "Project/Yektanet/product_recommender"

%cd drive/My\ Drive
%cp -r $FOLDERNAME ../../
%cd ../../
%cd product_recommender/

/content/drive/My Drive
/content
/content/product_recommender


In [5]:
%cd code

from utils import *
from crawler_functions import *
from clean_data import *
from TermFrequency import TermFrequency
from recommend import *

%cd ..

/content/product_recommender/code
/content/product_recommender


# Digikala Crawler and creating files

In [6]:
run_crawler(4322796, 6, 2, 1000)

/content/product_recommender/code/crawler_functions.py:32: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 32 of the file /content/product_recommender/code/crawler_functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = bs(res.content)


0 in 43.28174185752869 seconds done!
1 in 136.7251844406128 seconds done!
2 in 105.71302771568298 seconds done!


In [7]:
all_products = []

for file_name in range(3):
    path = 'data/pickle_files/digikala_p_'+str(1000+file_name)+'.pkl'
    products = read_pickle(path)
    all_products = all_products + products

columns_names=['dkp', 'product_name', 'description', 'classes', 'stocked']
all_products = pd.DataFrame(all_products, columns = columns_names)
all_products

,dkp,product_name,description,classes,stocked
0,4322796,کفش کوهنوردی زنانه سالومون مدل RS408144,,"[مد و پوشاک, زنانه, کفش زنانه, کفش ورزشی زنانه]",True
1,4322797,کاور طرح Harley Quinn مدل CHL50353 مناسب برای ...,,"[کالای دیجیتال, لوازم جانبی کالای دیجیتال, لوا...",True
2,4322798,مگنت و کاغذ یادداشت مدل S15,وایت برد یاددا...,"[خانه و آشپزخانه, دکوراتیو, لوازم تزیینی, سایر...",True
3,4322799,قارچ کش دیفوربین کاوش مدل اورتیواتاب حجم 250 م...,دیفوربین قارچ ...,"[خودرو، ابزار و تجهیزات صنعتی, باغبانی, سموم گ...",True
4,4322801,تابلو فرش دستبافت طرح گل کد ۴۳۲۲۸۰۱,,"[کتاب، لوازم تحریر و هنر, فرش, تابلو فرش]",False


In [8]:
all_products.to_csv('data/digikala_products_test_.csv', index=False)

# Read file

In [9]:
## csv
# data = pd.read_csv('data/digikala_products.csv') 
# data = pd.read_csv('data/digikala_products_test.csv') # the one we create in last section 
# data.head()

In [10]:
## pickle
data = read_pickle('data/digikala_products.pkl')
data

,dkp,product_name,description,classes,stocked
0,3270000,برچسب بدنه خودرو طرح حسین علیه السلام کد MH104,راهنمایی نصب: ...,"[خودرو، ابزار و تجهیزات صنعتی, لوازم جانبی خود...",True
1,3270001,آباژور مدل الگا,آباژور آینه‎ای...,"[خانه و آشپزخانه, دکوراتیو, چراغ خواب و آباژور]",False
2,3270002,جعبه لوازم خیاطی ژانومه کد 096,,"[خانه و آشپزخانه, خواب و حمام, ملزومات خیاطی]",True
3,3270003,کفش زنانه لیانا کد 293-GH,,"[مد و پوشاک, زنانه, کفش زنانه, کفش پاشنه دار ز...",True
4,3270004,کیت آموزشی منظومه شمسی مدل GH3497-5,,"[اسباب بازی، کودک و نوزاد, سرگرمی و آموزشی, اس...",False
...,...,...,...,...,...
1877,260110,کمربند مردانه شهر چرم مدل 1-245203,,"[مد و پوشاک, مردانه, اکسسوری مردانه, کمربند و ...",False
1878,260111,عروسک مدل Elephant Color Shawl ارتفاع 38 سانتی...,,"[اسباب بازی، کودک و نوزاد, سرگرمی و آموزشی, اس...",False
1879,260112,عروسک مدل Elephant Color Shawl ارتفاع 47 سانتی...,,"[اسباب بازی، کودک و نوزاد, سرگرمی و آموزشی, اس...",False
1880,260113,دستگاه اتوماتیک خوشبو کننده هوا نووا مدل NW024...,,"[خانه و آشپزخانه, شستشو و نظافت, مواد شوینده, ...",False


# EDA

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1882 entries, 0 to 1881
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   dkp           1882 non-null   int64 
 1   product_name  1882 non-null   object
 2   description   1882 non-null   object
 3   classes       1882 non-null   object
 4   stocked       1882 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 60.8+ KB


Main product classes:

In [12]:
classes_set_ = {p_classes[0] for p_classes in data['classes'] }
print('size of main classes:' ,len(classes_set_))
classes_set_

size of main classes: 9


{'اسباب بازی، کودک و نوزاد',
 'خانه و آشپزخانه',
 'خودرو، ابزار و تجهیزات صنعتی',
 'خوردنی و آشامیدنی',
 'زیبایی و سلامت',
 'مد و پوشاک',
 'ورزش و سفر',
 'کالای دیجیتال',
 'کتاب، لوازم تحریر و هنر'}

Product subclasses:

In [13]:
classes_set_ = {p_classes[1] for p_classes in data['classes'] }
print('size of subclasses:', len(classes_set_))
classes_set_

size of subclasses: 53


{'آشپزخانه',
 'آلات موسیقی',
 'ابزار برقی',
 'ابزار سلامت',
 'ابزار غیر برقی',
 'انبارداری و صنعتی',
 'اکسسوری لوازم شخصی',
 'باغبانی',
 'بچگانه',
 'تبلت و کتابخوان',
 'تجهیزات سفر',
 'حیوانات خانگی',
 'خشکبار و شیرینی',
 'خواب و حمام',
 'خواب کودک',
 'دوربین',
 'دوچرخه و لوازم جانبی',
 'دکوراتیو',
 'زنانه',
 'زنانه و مردانه',
 'ساعت و مچ بند هوشمند',
 'سرو و پذیرایی',
 'سرگرمی و آموزشی',
 'شستشو و نظافت',
 'صبحانه',
 'صنایع دستی',
 'صوتی و تصویری',
 'عطر',
 'غذاخوری',
 'فرش',
 'لبنیات',
 'لوازم آرایشی',
 'لوازم بهداشتی',
 'لوازم تحریر',
 'لوازم جانبی خودرو',
 'لوازم جانبی کالای دیجیتال',
 'لوازم خانگی برقی',
 'لوازم شخصی برقی',
 'لوازم مصرفی خودرو',
 'لوازم ورزشی',
 'لپ تاپ',
 'ماشین های اداری',
 'محتوای آموزشی',
 'محصولات مذهبی',
 'مردانه',
 'موبایل',
 'نرم افزار و بازی',
 'نور و روشنایی',
 'نوشیدنی',
 'کالاهای اساسی و خوار و بار',
 'کامپیوتر و تجهیزات جانبی',
 'کتاب و مجلات',
 'گردش و سفر'}

Stocked or not:

In [14]:
print(data[data['stocked']].shape[0], 'products are in stock.')
print(data[data['stocked']==False].shape[0], 'products are not!')

952 products are in stock.
930 products are not!


Having description:

In [15]:
print(data[data['description']!=''].shape[0], 'products have description.')
print(data[data['description']==''].shape[0], 'products have not.')

826 products have description.
1056 products have not.


# Cleaning and Preprocessing

In [16]:
class_string = lambda class_list: ' '.join(str(cls) for cls in class_list)
data['classes_string'] = pd.Series([class_string(cls_list) for cls_list in data['classes']])
data['product'] = data['product_name'].str.cat(data['classes_string'], sep=' ')

In [17]:
corpus = [clean_string(text, False) for text in data['product']]

In [18]:
# write_to_pickle(corpus, 'data/corpus_cleaned.pkl')
# corpus = read_pickle('data/corpus_cleaned.pkl')

In [19]:
# An example for texts
context = [
               ' فاطمه ساک خرید آلما کفش کوهنوردی کاغذ خرید',
               'دستبافت فرش مناسب',
               'ساک لباس'
]

In [20]:
context_ = [clean_string(text, False) for text in context]

Term Frequency

In [21]:
product_tf = TermFrequency(corpus)
products, word2index = product_tf.corpus_TF, product_tf.word2index
texts = TermFrequency(context_, data_type = 'text', word2index = word2index).corpus_TF

# Algorithm

## Jaccard

In [22]:
similarity_matrix = calculate_similarity(products[:10], texts, jaccard_sim)
similarity_matrix

array([[0.24845573, 0.33196065, 0.33161748, 0.24905596, 0.19890185,
        0.33184626, 0.36226321, 0.24854152, 0.19903912, 0.24888469],
       [0.24871311, 0.33230382, 0.33196065, 0.24914207, 0.19910776,
        0.33218943, 0.33207504, 0.2487989 , 0.19924502, 0.24914207],
       [0.24871311, 0.33230382, 0.33196065, 0.24914207, 0.19910776,
        0.33218943, 0.33207504, 0.27397013, 0.19924502, 0.28502771]])

In [23]:
pred = predict (similarity_matrix, 3)
pred

[array([6, 1, 5]), array([1, 5, 6]), array([1, 5, 6])]

## Cosine

In [24]:
similarity_matrix = calculate_similarity(products[:10], texts, cosine_sim)
similarity_matrix

array([[0.        , 0.        , 0.        , 0.2409658 , 0.        ,
        0.        , 0.11547005, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.12309149, 0.        , 0.13867505]])

In [25]:
pred = predict (similarity_matrix, 3)
pred

[array([3, 6, 9]), array([9, 8, 7]), array([9, 7, 8])]

# Evaluation
metric: nDCG

In [26]:
#ideal
true_relevance = np.array([[1, 0, 0]])
#predicted
scores = np.array([[1, 1, 0]])
ndcg_score(true_relevance, scores, k=2)

0.8154648767857287

# Enter your text here: 

In [27]:
def recommender(your_text, word2index, products, top = 5):
    your_text_cleaned = [clean_string(your_text, False)]
    your_text_tf = TermFrequency(your_text_cleaned, data_type = 'text', word2index = word2index).corpus_TF
    your_text_similarity_matrix = calculate_similarity(products, your_text_tf, jaccard_sim)
    return  predict (your_text_similarity_matrix, top)[0].tolist()

In [29]:
# your_text = input()
your_text = 'کتاب ویلون کتاب  '
pred_indices = recommender(your_text, word2index, products, top=10)
data[['dkp', 'product_name', 'classes', 'stocked']].iloc[pred_indices]

,dkp,product_name,classes,stocked
980,4135207,تابلو مدل Tgirl0,"[خانه و آشپزخانه, دکوراتیو, تابلو]",True
1632,777304,تابلو شاسی مدل No U-turn,"[خانه و آشپزخانه, دکوراتیو, تابلو شاسی]",False
655,3271511,پرده مدل fp01ab سایز 140x280 سانتی متر,"[خانه و آشپزخانه, دکوراتیو, پرده]",False
1190,4135444,تابلو شاسی مدل Peaky blinders کد S-8007,"[خانه و آشپزخانه, دکوراتیو, تابلو شاسی]",False
70,3270079,قاب عکس مدل 2020,"[خانه و آشپزخانه, دکوراتیو, قاب عکس]",True
215,3271013,مجسمه طرح کرگدن,"[خانه و آشپزخانه, دکوراتیو, مجسمه و تندیس]",True
1275,275154,کیف آدیداس مدل Festival,"[ورزش و سفر, تجهیزات سفر, کیف و کوله]",False
673,3271530,پرده مدل per09 سایز 140x280 سانتی متر,"[خانه و آشپزخانه, دکوراتیو, پرده]",False
1226,4135491,تابلو شاسی مدل 8016,"[خانه و آشپزخانه, دکوراتیو, تابلو شاسی]",False
1224,4135488,تابلو شاسی مدل Peaky blinders کد S-8015,"[خانه و آشپزخانه, دکوراتیو, تابلو شاسی]",False
